# buy-open-sell-close

On the first trading day of each week, buy on the open, then sell on the close.

In [1]:
import pandas as pd
import datetime
import pinkfish as pf

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# Fetch timeseries, select.
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)

# Add calendar columns.
ts = pf.calendar(ts)

# Finalize timeseries.
ts, start = pf.finalize_timeseries(ts, start)

# Create tradelog and daily balance objects.
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [4]:
ts

,open,high,low,close,adj_close,volume,dotw,dotm,doty,month,first_dotw,first_dotm,first_doty,last_dotw,last_dotm,last_doty
date,,,,,,,,,,,,,,,,
2015-01-02,206.38,206.88,204.18,205.43,173.70,121465900,4,2,2,1,False,True,True,True,False,False
2015-01-05,204.17,204.37,201.35,201.72,170.57,169632600,0,5,5,1,True,False,False,False,False,False
2015-01-06,202.09,202.72,198.86,199.82,168.96,209151400,1,6,6,1,False,False,False,False,False,False
2015-01-07,201.42,202.72,200.88,202.31,171.07,125346700,2,7,7,1,False,False,False,False,False,False
2015-01-08,204.01,206.16,203.99,205.90,174.10,147217800,3,8,8,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-14,542.85,544.96,540.12,543.75,543.75,42446900,2,14,227,8,False,False,False,False,False,False
2024-08-15,549.50,553.36,548.88,553.07,553.07,60846800,3,15,228,8,False,False,False,False,False,False
2024-08-16,551.42,555.02,551.26,554.31,554.31,44430700,4,16,229,8,False,False,False,True,False,False


Algorithm

In [5]:
pf.TradeLog.cash = capital

# Loop through timeseries.
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # On first trading day of the week, buy open, then sell close.
    if row.first_dotw:
        tlog.buy(date, row.open)
        tlog.sell(date, row.close)

    # Record daily balance.
    dbal.append(date, row.close, row.high, row.low)

Retrieve logs

In [6]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [7]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
498,2024-07-22,553.00,2024-07-22,554.65,1.65,34.65,21,1723.65,LONG,SPY
499,2024-07-29,546.02,2024-07-29,544.76,-1.26,-26.46,21,1697.19,LONG,SPY
500,2024-08-05,511.64,2024-08-05,517.38,5.74,126.28,22,1823.47,LONG,SPY
501,2024-08-12,534.21,2024-08-12,533.27,-0.94,-20.68,22,1802.79,LONG,SPY
502,2024-08-19,554.73,2024-08-19,559.61,4.88,102.48,21,1905.27,LONG,SPY


Get stats

In [8]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [9]:
pf.summary(stats)

,strategy
annual_return_rate,1.83
max_closed_out_drawdown,-13.98
best_month,6.04
worst_month,-5.08
sharpe_ratio,0.38
sortino_ratio,0.23
monthly_std,1.39
annual_std,6.10
